In [1]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
from geopandas.tools import overlay
import matplotlib.pyplot as plt
import os 
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
import xgboost as xgb
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, mean_squared_error
#import tensorflow as tf
#from keras.models import Sequential
#from keras.layers import Dense
#from tensorflow.keras.utils import to_categorical

## Espacios Públicos GDL

In [2]:
schema = 'espacios_publicos'
table = 'ep_amg'
gdf_gdl = aup.gdf_from_db(table,schema)

In [10]:
gdf_gdl.shape

(8499, 4)

## Espacios Públicos QRO

In [3]:
schema = 'espacios_publicos'
table = 'ep_qro'
gdf_qro = aup.gdf_from_db(table,schema)

In [11]:
gdf_qro.shape

(4814, 9)

## Espacios Públicos CDMX

In [4]:
schema = 'espacios_publicos'
table = 'ep_cdmx'
gdf_cdmx = aup.gdf_from_db(table,schema)

In [12]:
gdf_cdmx.shape

(11609, 13)

## Juntamos en un dataset

In [5]:
gdf_gdl = gdf_gdl.set_crs("EPSG:4326", allow_override=True)
gdf_qro = gdf_qro.set_crs("EPSG:4326", allow_override=True)
gdf_cdmx = gdf_cdmx.set_crs("EPSG:4326", allow_override=True)
print("CRS of gdf_gdl:", gdf_gdl.crs)
print("CRS of gdf_qro:", gdf_qro.crs)
print("CRS of gdf_cdmx:", gdf_cdmx.crs)

CRS of gdf_gdl: EPSG:4326
CRS of gdf_qro: EPSG:4326
CRS of gdf_cdmx: EPSG:4326


In [20]:
geodataframes = [gdf_gdl, gdf_qro, gdf_cdmx]
gdf = pd.concat(geodataframes)
gdf = gdf['geometry']
gdf.head()

0    POLYGON ((672507.02800 2272132.20130, 672520.1...
1    POLYGON ((672520.12560 2272170.35300, 672507.0...
2    POLYGON ((672494.32530 2272095.20510, 672480.3...
3    POLYGON ((672480.36630 2272060.73140, 672494.3...
4    POLYGON ((672468.71720 2272020.61270, 672456.8...
Name: geometry, dtype: geometry

In [21]:
gdf.shape

(24922,)

## Hexágonos 3 ciudades

In [23]:
hex_schema = "raster_analysis"
hex_table = "ndvi_analysis_hex"
c1 = 'Guadalajara'
c2 ='ZMVM'
c3 = 'Queretaro'
n = '11'
query = f'SELECT hex_id, geometry, city FROM {hex_schema}.{hex_table} WHERE \"city\" IN (\'{c1}\',\'{c2}\',\'{c3}\') AND \"res\" = {n}'
hex_gdl = aup.gdf_from_query(query, geometry_col='geometry')
#query = 'fSelect geometry, hex_id, city from raster_analysis.ndvi_analysis where city = guadalajara and res = 11'
hex_gdl.head(5)

,hex_id,geometry,city
0,8b49ab4b2758fff,"POLYGON ((-103.39247 20.75090, -103.39230 20.7...",Guadalajara
1,8b49ab4b270cfff,"POLYGON ((-103.39091 20.74832, -103.39074 20.7...",Guadalajara
2,8b49ab4b2792fff,"POLYGON ((-103.38489 20.74954, -103.38472 20.7...",Guadalajara
3,8b49ab4b2722fff,"POLYGON ((-103.39016 20.74679, -103.38998 20.7...",Guadalajara
4,8b49ab4b26f4fff,"POLYGON ((-103.38863 20.75304, -103.38845 20.7...",Guadalajara


In [29]:
hex_gdl.shape

(807141, 4)

In [24]:
hex_gdl['city'].unique()

array(['Guadalajara', 'Queretaro', 'ZMVM'], dtype=object)

In [27]:
hex_gdl = hex_gdl.set_crs("EPSG:4326", allow_override=True)
print("CRS of gdf_gdl:", gdf_gdl.crs)

CRS of gdf_gdl: EPSG:4326


## Dummy Variables
1 = Hexágonos que cuentan con espacios públicos

0 = Hexágonos que no cuentan con espacios públicos

In [28]:
gdf_sindex = gdf.sindex

# Create a Series of boolean values indicating which hexagons intersect with public spaces
intersects_public_spaces = hex_gdl['geometry'].apply(lambda geom: any(gdf_sindex.query(geom)))

hex_gdl['EspPublico'] = intersects_public_spaces.astype(int)
hex_gdl.head()

,hex_id,geometry,city,EspPublico
0,8b49ab4b2758fff,"POLYGON ((-103.39247 20.75090, -103.39230 20.7...",Guadalajara,0
1,8b49ab4b270cfff,"POLYGON ((-103.39091 20.74832, -103.39074 20.7...",Guadalajara,0
2,8b49ab4b2792fff,"POLYGON ((-103.38489 20.74954, -103.38472 20.7...",Guadalajara,0
3,8b49ab4b2722fff,"POLYGON ((-103.39016 20.74679, -103.38998 20.7...",Guadalajara,0
4,8b49ab4b26f4fff,"POLYGON ((-103.38863 20.75304, -103.38845 20.7...",Guadalajara,0


In [30]:
contador_ceros = np.sum(hex_gdl['EspPublico'] == 0)
contador_uno = np.sum(hex_gdl['EspPublico'])
print(f'Números de hexagonos sin Espacios Públicos: {contador_ceros} \nNúmero de Hexágonos con Espacios Públicos: {contador_uno}\nNúmero total de Hexágonos: {contador_ceros+contador_uno}')

Números de hexagonos sin Espacios Públicos: 713076 
Número de Hexágonos con Espacios Públicos: 94065
Número total de Hexágonos: 807141


## Hexágonos con info de humedad, vegetación y temperatura

select {ndmi}.hex_id, {ndmi}.geometry, {ndmi}.city, {ndmi}.res,

ndvi_mean, ndvi_std, ndvi_median, ndvi_max, ndvi_min, ndvi_diff, ndvi_tend,

ndmi_mean, ndmi_median, ndmi_diff

from raster_analysis.ndmi_analysis_hex ndmi

inner join raster_analysis.ndvi_analysis_hex ndvi

on ndmi.hex_id = ndvi.hex_id 

where ndmi.city  in ('Queretaro','Guadalajara','ZMVM') and ndmi.res = 11

In [31]:
hex_schema = "raster_analysis"
hex_tablee = "ndvi_analysis_hex"
hex_table = "ndmi_analysis_hex"
ndmi = "ndmi"
ndvi = "ndvi"
hex_id = "hex_id"
c1 = 'Guadalajara'
c2 ='ZMVM'
c3 = 'Queretaro'
n = '11'
query = f'SELECT {ndmi}.hex_id, {ndmi}.geometry, {ndmi}.city, {ndmi}.res, ndvi_mean, ndvi_std, ndvi_median, ndvi_max, ndvi_min, ndvi_diff, ndvi_tend,ndmi_mean, ndmi_median, ndmi_diff FROM {hex_schema}.{hex_table} {ndmi} INNER JOIN {hex_schema}.{hex_tablee} {ndvi} ON {ndmi}.{hex_id} = {ndvi}.{hex_id} WHERE {ndmi}.\"city\" IN (\'{c1}\',\'{c2}\',\'{c3}\') AND {ndmi}.\"res\" = {n}'
intt_gdl = aup.gdf_from_query(query, geometry_col='geometry')
intt_gdl.head(10)

,hex_id,geometry,city,res,ndvi_mean,ndvi_std,ndvi_median,ndvi_max,ndvi_min,ndvi_diff,ndvi_tend,ndmi_mean,ndmi_median,ndmi_diff
0,8b49ab4b2670fff,"POLYGON ((-103.39190 20.75208, -103.39172 20.7...",Guadalajara,11,0.264131,0.056046,0.278143,0.308765,0.173648,0.135117,-0.000757,0.053463,0.050777,0.097573
1,8b49ab4b2676fff,"POLYGON ((-103.39147 20.75180, -103.39130 20.7...",Guadalajara,11,0.409427,0.105657,0.433788,0.520438,0.244854,0.275584,-0.002333,0.091292,0.089714,0.098885
2,8b49ab4b270efff,"POLYGON ((-103.39044 20.74853, -103.39026 20.7...",Guadalajara,11,0.316967,0.086185,0.321497,0.407292,0.196025,0.211267,-0.002106,0.036697,0.035678,0.129225
3,8b49ab4b268efff,"POLYGON ((-103.38678 20.75338, -103.38660 20.7...",Guadalajara,11,0.085069,0.019698,0.090198,0.101235,0.063335,0.037900,-0.000685,-0.058275,-0.071303,0.174109
4,8b49ab4b2663fff,"POLYGON ((-103.39322 20.75244, -103.39305 20.7...",Guadalajara,11,0.178388,0.036286,0.191513,0.209757,0.112014,0.097743,-0.000920,0.001050,-0.010292,0.128643
5,8b49ab59516efff,"POLYGON ((-103.43305 20.61173, -103.43288 20.6...",Guadalajara,11,0.069822,0.019254,0.073560,0.082880,0.038931,0.043949,-0.000442,-0.074391,-0.088315,0.145272
6,8b49ab5951b5fff,"POLYGON ((-103.42548 20.61037, -103.42530 20.6...",Guadalajara,11,0.075444,0.033387,0.082717,0.102821,0.043347,0.059474,-0.001355,-0.076039,-0.093632,0.151256
7,8b49ab492b71fff,"POLYGON ((-103.37829 20.79758, -103.37812 20.7...",Guadalajara,11,0.324421,0.156561,0.275693,0.570776,0.145903,0.424873,-0.002101,-0.037157,-0.077201,0.326381
8,8b49ab492af0fff,"POLYGON ((-103.37420 20.80215, -103.37403 20.8...",Guadalajara,11,0.268156,0.108509,0.245064,0.420108,0.132961,0.287147,-0.001582,0.018301,0.001772,0.233537
9,8b49ab492a2afff,"POLYGON ((-103.37625 20.79986, -103.37607 20.8...",Guadalajara,11,0.547411,0.160259,0.590196,0.692978,0.285048,0.407930,-0.003809,0.151968,0.130599,0.186123


In [33]:
intt_gdl.shape

(799837, 14)

In [34]:
intt_gdl['city'].unique()

array(['Guadalajara', 'Queretaro', 'ZMVM'], dtype=object)

## Intersección

In [35]:
intt_gdl = intt_gdl.set_crs("EPSG:4326")
hex_gdl = hex_gdl.set_crs("EPSG:4326")

In [ ]:
inter = gpd.sjoin(hex_gdl, intt_gdl, op='intersects')
inter.drop(columns=['index_right','city_right','res','hex_id_right'], inplace=True)
inter.rename(columns={"hex_id_left": "hex_id","city_left": "city"}, inplace=True)
inter.head()

In [ ]:
pd.isna(inter).sum()